In [6]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})
    
add_message("1", "1")

In [7]:
add_message("2", "2")
add_message("3", "3")
add_message("4", "4")

In [8]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='1', additional_kwargs={}, response_metadata={}),
  AIMessage(content='1', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='2', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='3', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='4', additional_kwargs={}, response_metadata={}),
  AIMessage(content='4', additional_kwargs={}, response_metadata={})]}

In [ ]:
add_message("5", "5")